# Embeddings
* Transform the small parts of text in numbers (vectors) that are easily stored and searched by vector databases.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 003-embeddings.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **003-embeddings**.

## Track operations
From now on, we can track the operations **and the cost** of this project from LangSmith:
* [smith.langchain.com](https://smith.langchain.com)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [2]:
#pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [6]:
google_api_key = os.environ["GOOGLE_API_KEY"]

from langchain_google_genai import ChatGoogleGenerativeAI

chatModel = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

## Reminder: Steps of the RAG process.
* When you load a document, you end up with strings. Sometimes the strings will be too large to fit into the context window. In those occassions we will use the RAG technique:
    * Split document in small chunks.
    * **Transform text chunks in numeric chunks (embeddings)**.
    * Load embeddings to a vector database (aka vector store).
    * Load question and retrieve the most relevant embeddings to respond it.
    * Sent the embeddings to the LLM to format the response properly.

## Splitters: divide the loaded document in small chunks of text
* Also called "Document Transformers".
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/).
* See the list of built-in splitters [here](https://python.langchain.com/v0.1/docs/integrations/document_transformers/).

#### Simple splitting by character

In [7]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/be-good.txt")

loaded_data = loader.load()

In [8]:
#loaded_data

In [9]:
#loaded_data[0].page_content

In [10]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [11]:
texts = text_splitter.create_documents([loaded_data[0].page_content])

In [12]:
len(texts)

2

In [13]:
texts[0]

Document(metadata={}, page_content='Be good')

In [14]:
# texts[1]

#### Splitting with metadata

In [15]:
metadatas = [{"chunk": 0}, {"chunk": 1}]

documents = text_splitter.create_documents(
    [loaded_data[0].page_content, loaded_data[0].page_content], 
    metadatas=metadatas
)

In [16]:
documents[0]

Document(metadata={'chunk': 0}, page_content='Be good')

In [17]:
print(documents[0])

page_content='Be good' metadata={'chunk': 0}


## Embeddings: transform chunks of text in chunks of numbers
* Vector databases work very fast with numbers.
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/).
* See the list of embedding model providers [here](https://python.langchain.com/v0.1/docs/integrations/text_embedding/).
* **The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query**.

In [23]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001", google_api_key=google_api_key)

In [24]:
chunks_of_text =     [
        "Hi there!",
        "Hello!",
        "What's your name?",
        "Bond, James Bond",
        "Hello Bond!"
    ]

In [25]:
embeddings = embeddings_model.embed_documents(chunks_of_text)

In [26]:
len(embeddings)

5

In [27]:
len(embeddings[0])

3072

In [28]:
print(embeddings[0][:5])

[-0.020293528214097023, 0.00774721521884203, 0.0071714180521667, -0.0875835195183754, -0.015736732631921768]


In [31]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query

[-0.023928752169013023,
 0.0053334166295826435,
 0.020249219611287117,
 -0.11056268960237503,
 -0.008057850413024426,
 0.013581695035099983,
 -0.001066709402948618,
 0.005084080155938864,
 -0.010516899637877941,
 0.005776525009423494,
 -0.010842078365385532,
 -0.00748723978176713,
 0.0053252833895385265,
 -0.004669511690735817,
 0.16100965440273285,
 -0.008454704657196999,
 0.00720042921602726,
 0.02197089232504368,
 0.01779615506529808,
 -0.013412107713520527,
 0.0010607035364955664,
 0.008303876966238022,
 0.030080270022153854,
 -0.01131998561322689,
 0.0023292540572583675,
 -0.0061966776847839355,
 0.015345570631325245,
 0.015300636179745197,
 0.013187223114073277,
 0.009827186353504658,
 0.0025874662678688765,
 -0.006919818930327892,
 -0.0025023396592587233,
 -0.00010183897393289953,
 0.021187182515859604,
 0.013983067125082016,
 0.006912627723067999,
 -0.016882292926311493,
 -0.021305374801158905,
 0.014166999608278275,
 0.0005735944723710418,
 0.004807454999536276,
 0.00061441282

In [30]:
len(embedded_query)

3072

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 003-embeddings.py
* In terminal, make sure you are in the directory of the file and run:
    * python 003-embeddings.py